1. Engineer your features. Here you do not have them for free. You need to think of possible ways for transforming the collected data into meaningful features. For some ideas, consider traditional features such as texture features, color features, bags of visual words or more powerful ones involving CNNs. If you cannot think of anything, talk to the professor for some ideas.

2. Propose classification techniques to solve the problem. Suggestions here are the CNN directly, or SVMs/Random Forests allied with CNNs through the use of transfer learning.

3. Consider using data augmentation in the training (what about in the testing as well?)

4. Observation: You are free to use any solution to help you extract the features at this point.

5. Report all of your results for the validation and test data. The labels for the test will be released one week before the deadline.

In [10]:
#imports
from glob import glob
from skimage import io

In [11]:
#Definicoes
dataFolder = 'data/'
trainFolder = dataFolder + 'train/'
validationFolder = dataFolder + 'val/'
testFolder = dataFolder + 'test/'

In [12]:
#Funcoes

#Retorna o vetor da imagem dado o nome do seu arquivo
def le_imagem(name):
    return io.imread(name,plugin='matplotlib') 

#Retorna o vetor da imagem e a classe dado o caminho do arquivo
def le_imagem_e_classe(name):
    img = le_imagem(name)
    classe = int(name.split('/')[2].split('_')[0])
    return img, classe

#Retorna o caminho de todas as imagens dado a pasta
def nome_das_imagens(pasta):
    return glob(pasta + *)

In [13]:
print(nome_das_imagens())

(array([[[205, 181, 181],
        [206, 182, 182],
        [206, 182, 182],
        ...,
        [184, 172, 176],
        [184, 172, 176],
        [184, 172, 176]],

       [[206, 182, 182],
        [206, 182, 182],
        [206, 182, 182],
        ...,
        [184, 172, 176],
        [184, 172, 176],
        [184, 172, 176]],

       [[208, 184, 184],
        [207, 183, 183],
        [207, 183, 183],
        ...,
        [184, 172, 176],
        [184, 172, 176],
        [184, 172, 176]],

       ...,

       [[191, 170, 165],
        [185, 164, 159],
        [191, 170, 165],
        ...,
        [172, 148, 144],
        [170, 146, 142],
        [169, 145, 141]],

       [[193, 172, 167],
        [187, 166, 161],
        [193, 172, 167],
        ...,
        [175, 151, 147],
        [174, 150, 146],
        [173, 149, 145]],

       [[188, 167, 162],
        [183, 162, 157],
        [191, 170, 165],
        ...,
        [176, 152, 148],
        [175, 151, 147],
        [176, 152, 148]